In [1]:
import requests
import pandas as pd
import json
import time
import pymysql
import traceback2

In [2]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'defiplus'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()


def insert_project_tvl_df(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        dt = row["timestamp"].timestamp()
#         print(dt)
        try:
            print(dt, row["project"], row["tvlUSD"], row["tvlETH"], row["BTC"], row["ETH"], row["DAI"])
            sql = "INSERT INTO `project_tvl` (`timestamp_dt`, `project`, `tvlUSD`, `tvlETH`, `BTC`, `ETH`, `DAI`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s,%s,%s)"
            cursor.execute(sql, (dt, row["project"], row["tvlUSD"], row["tvlETH"], row["BTC"], row["ETH"], row["DAI"]))
#             connection.commit()
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()

In [3]:
api_key = "3a2957dbae161d9aac23aadb136880f304a1653c2003ca11cc478e2f30f8"
payload_projects = {'api-key': api_key} 
url_projects = 'https://data-api.defipulse.com/api/v1/defipulse/api/GetProjects'


In [4]:
res_proj = requests.get(url_projects,params= payload_projects).json()
projects = []
for res in res_proj:
    projects.append({"name":res['name'], "category":res['category']})

In [5]:
res_proj

[{'category': 'Lending',
  'variability': 'medium',
  'website': 'https://aave.com',
  'chain': 'Multichain,Ethereum-Polygon',
  'id': 22,
  'name': 'Aave',
  'value': {'tvl': {'USD': {'value': 11867702865, 'relative_1d': -12.16},
    'ETH': {'value': 5111027.16862333, 'relative_1d': -1.63},
    'BTC': {'value': 335471.95553274045, 'relative_1d': -2.7800000000000002}},
   'total': {'USD': {'value': 11867702865, 'relative_1d': -12.16},
    'ETH': {'value': 1127073.139198, 'relative_1d': -0.31},
    'BTC': {'value': 61535.69738088999, 'relative_1d': 0.77}},
   'balance': {'ERC20': {'DAI': {'value': 601554531.0961745,
      'relative_1d': 7.42}}}},
  'contributesTo': None,
  'relative_1d': -5.65,
  'timestamp': 1623484800},
 {'category': 'Lending',
  'chain': 'Ethereum',
  'id': 0,
  'name': 'Maker',
  'value': {'tvl': {'USD': {'value': 7647090024, 'relative_1d': -4.39},
    'ETH': {'value': 3293348.7902565915, 'relative_1d': 0.97},
    'BTC': {'value': 216165.18998398355, 'relative_1d': 

In [6]:
df_proj = pd.DataFrame(projects)
df_proj

,name,category
0,Aave,Lending
1,Maker,Lending
2,Curve Finance,DEXes
3,Compound,Lending
4,Uniswap,DEXes
...,...,...
99,Primitive,Derivatives
100,QIAN,Lending
101,UniTrade,DEXes
102,Robo-Advisor for Yield,Lending


In [7]:
url = f"https://data-api.defipulse.com/api/v1/defipulse/api/GetHistory"

In [40]:
def get_historical_tvl_by_project_name(pname,retry=0):
    pname = pname.replace(" ","-")
    payload = {'api-key': api_key
                ,'project': pname
                ,'length':'90'
                ,'resolution':'days'
              }
    try:
        res = requests.get(f'{url}',params= payload).json()
        df_res = pd.json_normalize(res)
        new_cns = []
        for cn in df_res.columns:
            if cn == 'timestamp':
                new_cn = cn
            else:
                new_cn = f'{cn}'
            new_cns.append(new_cn)
        df_res.columns = new_cns
        df_res['project'] = pname
        #display(df_res)
        return  df_res
    except:
        print(pname)
        return pd.DataFrame()

In [41]:
def get_slug(name):
    return name.replace(" ", "-", -1).lower()

In [42]:
maker_df = get_historical_tvl_by_project_name(get_slug('Aave'))
maker_df

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,project
0,1616198400,5526453789,3053137.572717,9329.373001,6.338702e+05,1.940556e+07,aave
1,1616284800,5522093480,3060230.139598,9158.001266,6.520777e+05,1.981879e+07,aave
2,1616371200,5477592106,3068679.05098,8907.736192,6.688788e+05,2.509299e+07,aave
3,1616457600,5084922764,3016845.206495,8492.205525,6.874597e+05,2.230034e+07,aave
4,1616544000,5065873191,3032912.166078,8598.830288,6.830692e+05,2.051189e+07,aave
...,...,...,...,...,...,...,...
81,1623196800,12624315295,3631075.6317610475[object Object],57945.475840,1.124759e+06,5.867496e+08,aave
82,1623283200,13283719794,3700367.972866225[object Object],60488.738226,1.162944e+06,5.877963e+08,aave
83,1623369600,12713268221,3674400.890975786[object Object],60858.341051,1.140554e+06,5.528557e+08,aave
84,1623456000,12126818226,3698862.0134353666[object Object],61240.615873,1.136154e+06,5.817268e+08,aave


In [45]:
tvls_df = pd.DataFrame
project_names = df_proj['name'].values
for project in project_names:
    df = get_historical_tvl_by_project_name(get_slug(project))
    if df is not None and not df.empty:
        if tvls_df.empty:
            tvls_df = df
        else:
            tvls_df = tvls_df.append(df)

tvls_df = tvls_df.astype(object).where(pd.notnull(tvls_df), None)
tvls_df            

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,project
0,1616198400,5526453789,3053137.572717,9329.373001,633870.189099,19405556.670555,aave
1,1616284800,5522093480,3060230.139598,9158.001266,652077.730463,19818785.417958,aave
2,1616371200,5477592106,3068679.05098,8907.736192,668878.782507,25092986.993949,aave
3,1616457600,5084922764,3016845.206495,8492.205525,687459.687656,22300338.975895,aave
4,1616544000,5065873191,3032912.166078,8598.830288,683069.204768,20511885.883608,aave
...,...,...,...,...,...,...,...
86,1616112000,12796,7.202116,0.0,None,12768.952584,connext
87,1616025600,12833,7.037295,0.0,None,12768.952584,connext
88,1615939200,12785,7.083573,0.0,None,12768.952584,connext
89,1615852800,12805,7.146724,0.0,None,12768.952584,connext


In [49]:
tvls_df['timestamp_dt'] = pd.to_datetime(tvls_df['timestamp'], unit='s')

In [50]:
tvls_df

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,project,timestamp_dt
0,1616198400,5526453789,3053137.572717,9329.373001,633870.189099,19405556.670555,aave,2021-03-20
1,1616284800,5522093480,3060230.139598,9158.001266,652077.730463,19818785.417958,aave,2021-03-21
2,1616371200,5477592106,3068679.05098,8907.736192,668878.782507,25092986.993949,aave,2021-03-22
3,1616457600,5084922764,3016845.206495,8492.205525,687459.687656,22300338.975895,aave,2021-03-23
4,1616544000,5065873191,3032912.166078,8598.830288,683069.204768,20511885.883608,aave,2021-03-24
...,...,...,...,...,...,...,...,...
86,1616112000,12796,7.202116,0.0,None,12768.952584,connext,2021-03-19
87,1616025600,12833,7.037295,0.0,None,12768.952584,connext,2021-03-18
88,1615939200,12785,7.083573,0.0,None,12768.952584,connext,2021-03-17
89,1615852800,12805,7.146724,0.0,None,12768.952584,connext,2021-03-16


In [39]:
tvls_df = tvls_df.astype(object).where(pd.notnull(tvls_df), None)
insert_project_tvl_df(tvls_df)

1614585600.0 Maker 5677030517 3943313.3635719544 13912.257609999999 2959253.60584 9846083.679046
1614556800.0 Maker 5571529635 3924275.676874965 13891.380536 2966932.279352 9993003.820215
1614470400.0 Maker 5700869533 3901898.9993497827 13801.967879 2957953.957274 10331195.531553
1614384000.0 Maker 5684942302 3933590.0181977954 13820.521112999999 2971511.063237 10342335.731899
1614297600.0 Maker 5741121261 3890017.522664752 13902.712941 2951888.870287 10383247.757215
1614211200.0 Maker 6228048928 3833093.6712600244 13831.247111 2950741.16608 10290456.795463
1614124800.0 Maker 5929426920 3760656.3835859704 13812.805412 2915731.786624 9821197.793251
1614038400.0 Maker 6686807099 3758068.8684441247 13945.448644 2890881.11696 10175052.732921
1613952000.0 Maker 7159530336 3695489.4191610278 13937.296425 2860040.351637 9250082.033816
1613865600.0 Maker 7054215434 3673630.4682251606 13836.880361 2853782.447114 8528125.460323
1613779200.0 Maker 7139395637 3646697.8434647582 13701.768639 283226

1606953600.0 Maker 2747490110 4619416.09361602 14989.234524 2541399.818253 305524617.021674
1606867200.0 Maker 2758962180 4711986.234458259 15109.381862 2549011.999749 345662226.202641
1606780800.0 Maker 2842998127 4636406.541202563 15118.637395 2555454.763264 358449427.521509
1606694400.0 Maker 2684315335 4637885.440063582 15108.991982 2550604.781082 343652765.102718
1614585600.0 Aave 4779076835 3319587.149049081 9703.620494 572381.258278 29518080.092067
1614556800.0 Aave 4632987481 3263218.7700738152 9692.381725 570071.462865 28626277.317868
1614470400.0 Aave 4590524702 3141935.390301496 9545.429259 556424.249613 34832325.987045
1614384000.0 Aave 4491782994 3108005.6420085384 9737.753164 558456.927257 42294309.972109
1614297600.0 Aave 4551947478 3084267.8018240216 9618.719525 554048.345105 44491524.084305
1614211200.0 Aave 4999904566 3077224.146823321 9694.127787 555675.350039 44541636.826513
1614124800.0 Aave 4915389915 3117517.5461406736 10074.445354 553655.537567 42928486.184948
1

1607040000.0 Aave 1850267185 3016461.280751235 14095.413554 401167.315284 14025028.414687
1606953600.0 Aave 1818596651 3057646.9072078285 14795.607089 409000.773531 16826633.888997
1606867200.0 Aave 1712788103 2925242.695381883 14615.66324 416309.355906 9103414.622452
1606780800.0 Aave 1769664475 2885996.9585283515 14285.11414 420882.029665 7899846.067676
1606694400.0 Aave 1630392979 2816947.6813296936 15323.198594 400035.560516 13391463.392294
1614585600.0 Compound 4621351477 3210029.782726477 38748.526435 1044571.544343 429937950.672923
1614556800.0 Compound 4548125991 3203447.0551360794 38492.32017 1038134.305455 426647907.27574
1614470400.0 Compound 4523713964 3096207.4973477977 37925.397126 1024933.71337 354321460.16082996
1614384000.0 Compound 4518202007 3126285.786345426 37975.090224 1028187.669647 346407046.744361
1614297600.0 Compound 4584221392 3106135.671405147 37882.175594 1114751.17706 339314377.75108796
1614211200.0 Compound 4853805198 2987306.329970889 37199.859431 11226

1607558400.0 Compound 1639081872 2861275.85231736 23527.943289 1068741.944906 320700611.931319
1607472000.0 Compound 1612532777 2888601.2772284322 23512.894238 1083780.157145 318456414.964923
1607385600.0 Compound 1673643724 2834186.3510126667 23566.118879 1093765.952006 312013173.625493
1607299200.0 Compound 1672865517 2809744.225536632 23640.12782 1091798.712364 308811670.107982
1607212800.0 Compound 1648373137 2775739.895596531 24439.561465 1077221.281239 301611897.961209
1607126400.0 Compound 1544435984 2705217.9572962467 22529.507606 1061723.724188 303695234.66221297
1607040000.0 Compound 1632562020 2661539.9990218296 22447.942786 1077004.883656 318561364.04681504
1606953600.0 Compound 1630194482 2740882.159490223 22909.680948 1117597.481233 307679941.60115296
1606867200.0 Compound 1647520225 2813772.7575488454 22827.187424 1119311.10872 348614344.744069
1606780800.0 Compound 1701777524 2775285.8396255644 22988.984117 1099322.830776 361604499.467765
1606694400.0 Compound 153752008

1608681600.0 Curve-Finance 1503608810 2389753.1906101494 23071.722513 721.157333 155389534.93784708
1608595200.0 Curve-Finance 1490719120 2458958.696225917 23149.741864 577.167805 169138896.0191002
1608508800.0 Curve-Finance 1504509718 2378784.318623808 23097.457814 154.778649 160359921.78910443
1608422400.0 Curve-Finance 1502078501 2311281.141423933 22891.775021 None 152023459.44287804
1608336000.0 Curve-Finance 1455535759 2265919.046951865 22715.229019 None 153685036.02705082
1608249600.0 Curve-Finance 1476967821 2309674.919855505 22588.833485000003 None 156640504.75244257
1608163200.0 Curve-Finance 1458019239 2306007.305423316 22791.396502 None 154451204.59442684
1608076800.0 Curve-Finance 1551058983 2640325.105115329 30100.07821 None 155802949.04925612
1607990400.0 Curve-Finance 1516591131 2591666.0360914594 28884.173945000002 None 158112348.50171074
1607904000.0 Curve-Finance 1511719357 2561715.1714905445 28425.264695 None 159886901.18812472
1607817600.0 Curve-Finance 1485504591 2

...

1614297600.0 Badger-DAO 1455103762 985936.1741628611 16115.788486999998 84433.488259 None
1614211200.0 Badger-DAO 1533510473 943809.1056800488 16219.156845000001 83317.052313 None
1614124800.0 Badger-DAO 1505391272 954773.4331198072 16440.284877 83067.652625 None
1614038400.0 Badger-DAO 1712021931 962177.6470786593 16727.826343 83603.866075 None
1613952000.0 Badger-DAO 1848459320 954107.5375380026 16789.971145 82837.182742 None
1613865600.0 Badger-DAO 2091950988 1089427.3019377887 19343.473158 155151.837802 None
1613779200.0 Badger-DAO 2084285735 1064622.3688175832 18974.382789 153988.513332 None
1613692800.0 Badger-DAO 2014476608 1037673.2607374288 19902.265457 153043.192168 None
1613606400.0 Badger-DAO 2064623314 1115138.574630695 20120.8356 157407.837523 None
1613520000.0 Badger-DAO 1960485457 1099506.161879017 20080.629785999998 158690.72415 None
1613433600.0 Badger-DAO 1930341866 1084742.0490688605 20270.659051 156859.132013 None
1613347200.0 Badger-DAO 1980129987 1097322.81173282

1613520000.0 Balancer 1413506208 792741.8079032674 3653.841118 260335.646597 33160787.79312857
1613433600.0 Balancer 1395630127 784264.5442080537 3529.531254 254607.304223 34452751.820895374
1613347200.0 Balancer 1424824676 789590.9005768879 3529.238351 255467.531907 35007683.74972626
1613260800.0 Balancer 1372914964 755573.5747502821 3531.5588930000004 246909.72631 35781487.86935067
1613174400.0 Balancer 1389418944 752705.4249959369 3503.876155 245467.128476 35804928.0155331
1613088000.0 Balancer 1265150246 707582.3947561228 3367.238736 235507.436279 38713053.950494125
1613001600.0 Balancer 1221687415 700725.8067291478 3385.5886450000003 234634.749691 57133228.07509235
1612915200.0 Balancer 1188297142 669987.8451294253 3406.975025 232066.010939 49088285.03263951
1612828800.0 Balancer 1133091281 647599.1501303095 3379.609143 229017.186395 38798970.81324691
1612742400.0 Balancer 1056161055 654026.3892845201 3525.523138 228086.677143 35838437.787743725
1612656000.0 Balancer 1097000103 65

1614124800.0 InstaDApp 1034269207 655970.82958077 194.925586 54875.872484 868753631.619813
1614038400.0 InstaDApp 1105178561 621124.1153923972 421.265276 57212.970865 867841482.3250719
1613952000.0 InstaDApp 1146450407 591756.0440184374 433.045772 58012.433458 870585775.8693719
1613865600.0 InstaDApp 1132852450 589956.6458184697 402.445233 55090.490891 866699487.755984
1613779200.0 InstaDApp 1145444210 585075.9844108347 429.67855 58519.824684 869007005.6754489
1613692800.0 InstaDApp 1140142166 587296.489023046 430.478311 59307.270036 870478582.3351749
1613606400.0 InstaDApp 1113892927 601632.7348834696 429.416378 59161.800196 871273284.807856
1613520000.0 InstaDApp 1101694133 617867.112155508 286.788804 59061.358315 871047943.5836049
1613433600.0 InstaDApp 526046212 295607.9728469155 286.641734 58645.678716 297869775.501637
1613347200.0 InstaDApp 322182483 178542.91913040105 299.152666 57958.030275 89304529.768343
1613260800.0 InstaDApp 316530133 174200.01265787953 299.140409 54487.812

1614384000.0 RenVM 991089107 685765.6615210036 17488.863665 None None
1614297600.0 RenVM 1053590832 713882.639274728 18146.049513 None None
1614211200.0 RenVM 1092345038 672290.9374019116 17885.095488 None None
1614124800.0 RenVM 1063855959 674735.8146762225 17880.268606 None None
1614038400.0 RenVM 1207103442 678407.1679068408 17786.965264 None None
1613952000.0 RenVM 1276376263 658819.0500523907 17823.693735 None None
1613865600.0 RenVM 1252096359 652055.409508236 17784.202018 None None
1613779200.0 RenVM 1306249340 667212.8697446585 17928.00232 None None
1613692800.0 RenVM 1151738157 593269.6781604459 18179.838061 None None
1613606400.0 RenVM 1126120176 608236.8824434902 18153.184684 None None
1613520000.0 RenVM 1069628252 599883.4879364688 17874.051996 None None
1613433600.0 RenVM 1044204143 586783.1816087303 18476.042582 None None
1613347200.0 RenVM 1054859288 584568.2695025243 18392.624587 None None
1613260800.0 RenVM 1020423692 561582.615778322 17846.3754 None None
1613174400.0 

KeyboardInterrupt: 